In [2]:
import duckdb
import pandas as pd
import torch
from transformers import *
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics.pairwise import cosine_similarity
import gensim
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import nltk


2024-06-24 20:50:31.928683: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
comet_ml is installed but `COMET_API_KEY` is not set.
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v4.40. Import as `from transformers import GenerationMixin` instead.
  warnings.w

In [3]:
# Replace 'your_database.duckdb' with your actual database file path
con = duckdb.connect(database='/Users/sheillapurwandiary/spurwand/BIPM/Semester2/2TWSM/OntologyProject/ontology/Duckdb/isrecon_AIS11.duckdb')


# Replace 'your_table' with your actual table name
ontology_df = con.execute("""
SELECT * from main.ontology
where level_3  in ('research method','data analysis method')""").df()

#filtering the ontology for level_3  research method and data analysis method.

synonyms_df = con.execute("""
SELECT  DISTINCT  synonym
FROM synonyms  s
join ontology o
on s.ent_id = o.ent_id 
where o.level_3  in ('research method', 'data analysis method')""").df()

In [4]:
synonyms_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31770 entries, 0 to 31769
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   synonym  31770 non-null  object
dtypes: object(1)
memory usage: 248.3+ KB


In [5]:
ontology_df

,ent_id,definition,label,parent,level,related,level_1,level_2,level_3,level_4,level_5,level_6,level_7,level_8,level_9,level_10
0,research method,A research method is a systematic plan for con...,,methodological entity,3.0,None,entity,methodological entity,research method,,,,,,,
1,conceptual method,A conceptual method refers to a systematic app...,CONCEPTUAL_METHOD,research method,4.0,None,entity,methodological entity,research method,conceptual method,,,,,,
2,literature study,A literature study is a systematic and compreh...,CONCEPTUAL_METHOD,conceptual method,5.0,None,entity,methodological entity,research method,conceptual method,literature study,,,,,
3,structured literature research,Structured literature research is a methodical...,CONCEPTUAL_METHOD,literature study,6.0,None,entity,methodological entity,research method,conceptual method,literature study,structured literature research,,,,
4,literature sample,A literature sample refers to a specific piece...,CONCEPTUAL_METHOD,structured literature research,7.0,None,entity,methodological entity,research method,conceptual method,literature study,structured literature research,literature sample,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894,questionable research practice,Questionable research practice refers to the a...,ANALYSIS_METHOD,quantitative analysis,5.0,None,entity,methodological entity,data analysis method,quantitative analysis,questionable research practice,,,,,
895,data dredging,Data dredging refers to the practice of conduc...,ANALYSIS_METHOD,questionable research practice,6.0,None,entity,methodological entity,data analysis method,quantitative analysis,questionable research practice,data dredging,,,,
896,p hacking,P hacking is the manipulative practice of rean...,ANALYSIS_METHOD,data dredging,7.0,None,entity,methodological entity,data analysis method,quantitative analysis,questionable research practice,data dredging,p hacking,,,
897,HARKing,HARKing refers to the practice of formulating ...,ANALYSIS_METHOD,questionable research practice,6.0,None,entity,methodological entity,data analysis method,quantitative analysis,questionable research practice,HARKing,,,,


In [6]:
synonyms_df

,synonym
0,QDA Miner software
1,QDA Miner Lite software
2,code each interview
3,coded each statement
4,annotate all interview
...,...
31765,galvanic skin conductance activity
31766,tonic skin conductance response
31767,skin conductance evaluation
31768,telemonitoring tool


In [7]:
print('Before ', synonyms_df.shape)
synonyms_df.drop_duplicates(inplace=True)
print('After: ', synonyms_df.shape)

#resetting index to avoid issues
synonyms_df.reset_index(drop=True, inplace=True)

Before  (31770, 1)
After:  (31770, 1)


In [8]:
# Creating a copy of the ontology_df and selecting the 'parent' column
data_ontology = ontology_df.copy()
data_ontology = data_ontology[['parent']]
data_ontology['text'] = data_ontology['parent']
del(data_ontology['parent'])

# Creating a copy of the synonyms_df and selecting the 'synonym' column
data_synonyms = synonyms_df.copy()
data_synonyms['text'] = data_synonyms['synonym']
del(data_synonyms['synonym'])


In [9]:
print('Before ', data_ontology.shape)
data_ontology.drop_duplicates(inplace=True)
print('After: ', data_ontology.shape)

#resetting index to avoid issues
data_ontology.reset_index(drop=True, inplace=True)

Before  (899, 1)
After:  (220, 1)


In [10]:
data_synonyms


,text
0,QDA Miner software
1,QDA Miner Lite software
2,code each interview
3,coded each statement
4,annotate all interview
...,...
31765,galvanic skin conductance activity
31766,tonic skin conductance response
31767,skin conductance evaluation
31768,telemonitoring tool


In [11]:
data_ontology

,text
0,methodological entity
1,research method
2,conceptual method
3,literature study
4,structured literature research
...,...
215,statistical test assumption
216,homoskedasticity test
217,odds ratio homogeneity test
218,questionable research practice


In [12]:
# Unique entries in data_synonyms
unique_synonyms = data_synonyms['text'].unique()
print(f"Unique entries in data_synonyms: {len(unique_synonyms)}")

# Unique entries in data_ontology
unique_ontology = data_ontology['text'].unique()
print(f"Unique entries in data_ontology: {len(unique_ontology)}")

# Find common entries between the two dataframes
common_entries = set(unique_synonyms).intersection(set(unique_ontology))
print(f"Common entries between data_synonyms and data_ontology: {len(common_entries)}")
print(f"Common entries: {common_entries}")


Unique entries in data_synonyms: 31770
Unique entries in data_ontology: 220
Common entries between data_synonyms and data_ontology: 4
Common entries: {'research method', 'quantitative analysis', 'archival database', 'qualitative analysis'}


In [13]:
data_synonyms_list = data_synonyms["text"].to_list()
data_ontology_list = data_ontology["text"].to_list()

In [14]:
data_synonyms_list

['QDA Miner software',
 'QDA Miner Lite software',
 'code each interview',
 'coded each statement',
 'annotate all interview',
 'labeled each statement',
 'label each statement',
 'axial coding',
 'annotated the interview',
 'annotated each interview',
 'constant comparative method',
 'narrative approach',
 'quantitative article',
 'quantitative paper',
 'SAS enterprise miner',
 'R software environment',
 'R implementation',
 'CRAN',
 'predictive modeling',
 'predictive analytics',
 'ResNet',
 'GPT 2',
 'generative pre trained transformer 2',
 'classification algorithm',
 'k nearest neighbor',
 'naive bayes',
 'classification and regression tree',
 'CART tree',
 'C4 5',
 'metalearning',
 'ensemble method',
 'booting algorithm',
 'clustering study',
 'clustering research',
 'clustering analysis',
 'density based spatial clustering of applications with noise',
 'self supervised learning',
 'cross industry standard process for data mining',
 'CRISP DM',
 'face recognition',
 'Face ID',
 '

In [15]:
data_ontology_list

['methodological entity',
 'research method',
 'conceptual method',
 'literature study',
 'structured literature research',
 'literature sample',
 'bibliometric software',
 'literature database',
 'Web of Science Core Collection',
 'academic journal list',
 'individual IS journal',
 'bibliometric analysis',
 'simulation',
 'stochastic simulation',
 'Monte Carlo simulation',
 'Markov chain Monte Carlo',
 'computational algorithm',
 'mathematical model',
 'economic model',
 'stochastic model',
 'Markov model',
 'optimization model',
 'discrete optimization',
 'integer programming',
 'integer linear programming',
 'combinatorial optimization',
 'traveling purchaser problem',
 'multiple criteria decision making',
 'multi objective optimization',
 'design science',
 'design methodology',
 'design process',
 'conceptual modelling',
 'maturity model',
 'capability maturity model',
 'data modeling',
 'database modeling',
 'method engineering',
 'business process modeling',
 'business process m

In [31]:

# Combine both lists to create a corpus
combined_data = data_synonyms_list + data_ontology_list

# Tokenize the text (simple whitespace tokenization)
tokenized_data = [text.split() for text in combined_data]

# Train the Word2Vec model
model = Word2Vec(tokenized_data, vector_size=100, window=5, min_count=1, workers=4)

# Get vectors for each word in data_synonyms_list and data_ontology_list
embeddings_synonyms = {word: model.wv[word] for text in data_synonyms_list for word in text.split()}
embeddings_ontology = {word: model.wv[word] for text in data_ontology_list for word in text.split()}

# Define the get_embeddings function
def get_embeddings(term):
    tokens = term.split()
    embeddings = {}
    for token in tokens:
        if token in model.wv:
            embeddings[token] = model.wv[token]
    return embeddings

# Define the is_synonym function
def is_synonym(new_term):
    """
    Compare a new_term with the list of synonyms and ontology terms stored using cosine similarity.

    Args:
        new_term: a single term to be compared
    
    Returns:
        A string that tells if the term is a synonym to x word and the score or if there was not a synonym in the list
    """
    new_term_embds = get_embeddings(new_term)
    if not new_term_embds:
        return f"No embeddings found for the new term: {new_term}"

    similarity_scores = {}
    overall_embeddings = {**embeddings_synonyms, **embeddings_ontology}
    
    for term, embedding in overall_embeddings.items():
        for new_term_token, new_term_embedding in new_term_embds.items():
            similarity = cosine_similarity(new_term_embedding.reshape(1, -1), embedding.reshape(1, -1))[0][0]
            similarity_scores[term] = similarity
    
    sorted_similarity_scores = dict(sorted(similarity_scores.items(), key=lambda item: item[1], reverse=True))
    
    if sorted_similarity_scores and list(sorted_similarity_scores.values())[0] > 0.7:
        top_synonym = list(sorted_similarity_scores.keys())[0]
        return f"'{new_term}' is synonym to '{top_synonym}' with a similarity score of {list(sorted_similarity_scores.values())[0]:.2f}"
    else:
        return f"No close synonyms found in the list for the term: {new_term}"

# Example usage
print(is_synonym("apple"))
print(is_synonym("integer programming"))
print(is_synonym("discrete optimization"))
print(is_synonym("questionable research practice"))

No embeddings found for the new term: apple
'integer programming' is synonym to 'programming' with a similarity score of 1.00
'discrete optimization' is synonym to 'optimization' with a similarity score of 1.00
'questionable research practice' is synonym to 'practice' with a similarity score of 1.00


In [17]:
combined_data = data_synonyms_list + data_ontology_list

In [18]:
tokenized_data = [text.split() for text in combined_data]

In [19]:
# Train the Word2Vec model
model = Word2Vec(tokenized_data, vector_size=100, window=5, min_count=1, workers=4)

# Get vectors for each word in data_synonyms_list and data_ontology_list
synonym_vectors = {word: model.wv[word] for text in data_synonyms_list for word in text.split()}
ontology_vectors = {word: model.wv[word] for text in data_ontology_list for word in text.split()}

# Example: Print vector for a specific word
print("Vector for 'research':", ontology_vectors.get('research'))
print("Vector for 'audio':", synonym_vectors.get('audio'))

Vector for 'research': [ 0.09319612 -0.08419403  0.40410915  0.11716118 -0.1370745  -0.24179965
  0.17730439  0.34169167  0.14157328 -0.4206886   0.14863181 -0.37313494
  0.17654973 -0.07061384  0.06729862 -0.16613318  0.03670017 -0.22738826
 -0.03036536 -0.6239756  -0.03152496  0.43196243  0.1667981  -0.04803292
 -0.17558493  0.13238981 -0.4719396  -0.10090797 -0.05838131  0.05471925
 -0.16750416 -0.14575857  0.39902765  0.1593343  -0.30448285  0.2424973
  0.03066631 -0.02267972 -0.29449704 -0.16376093  0.02703498 -0.22639926
 -0.02312569  0.30129087  0.07635976 -0.46758378 -0.15677613 -0.27619848
  0.04277194  0.261812   -0.3688721  -0.25070834  0.43569505  0.00415363
  0.13761587  0.00616774  0.03904133 -0.23057954  0.24122216  0.02288042
 -0.3582873  -0.23690954  0.39606732  0.2394026  -0.44352555  0.5966573
 -0.5458234   0.117043   -0.13415045 -0.15954468  0.23216367  0.43381035
  0.317684    0.02109584  0.22612023  0.12648651  0.49473417 -0.32439283
 -0.20937142 -0.19280468 -0.31

In [20]:
def is_synonym(new_term):
    """
    Compare a new_term with the list of synonyms and ontology terms stored using cosine similarity.

    Args:
        new term: a single term to be compared
    
    Returns:
        A string that tells if the term is a synonym to x word and the score or if there was not a synonym in the list
    """
    
    new_term_embds = get_embeddings(new_term)
    similarity_scores = {}
    overall_embeddings = embeddings_synonyms #| embeddings_ontology
    for term, embedding in overall_embeddings.items():
        similarity = cosine_similarity(new_term_embds[new_term[0]].reshape(1, -1), embedding.reshape(1, -1))[0][0]
        similarity_scores[term] = similarity
    sorted_similarity_scores = dict(sorted(similarity_scores.items(), key=lambda item: item[1], reverse=True))
    if sorted_similarity_scores and list(sorted_similarity_scores.values())[0] > 0.7:
        top_synonym = list(sorted_similarity_scores.keys())[0]
        return f"is synonym to {top_synonym} with a similarity score of {list(sorted_similarity_scores.values())[0]}"
    else:
        return "No close synonyms found in the list."

In [21]:
new_term = ["apple"]
is_synonym(new_term)

AttributeError: 'list' object has no attribute 'split'